In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import numpy as np
import pandas as pd
import xarray as xr
from eofs.xarray import Eof
import esem

import warnings
warnings.filterwarnings('ignore')

2024-01-25 15:36:33.635187: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-25 15:36:33.635230: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Utilities for normalizing the emissions data
min_co2 = 0.
max_co2 = 2400
def normalize_co2(data):
    return data / max_co2

def un_normalize_co2(data):
    return data * max_co2

min_ch4 = 0.
max_ch4 = 0.6
def normalize_ch4(data):
    return data / max_ch4

def un_normalize_ch4(data):
    return data * max_ch4

data_path = "../"


In [ ]:
# Get one combined historical + ssp585 timeseries for now
X = xr.open_mfdataset([data_path + 'inputs_historical.nc', data_path + 'inputs_ssp585.nc']).compute()
# Take the 2nd ensemble member for the historical (the first one has some missing DTR values for some reason...) and the 1st (only) one for SSP585
Y = xr.concat([xr.open_dataset(data_path + 'outputs_historical.nc').sel(member=2), xr.open_dataset(data_path + 'outputs_ssp585.nc').sel(member=1)], dim='time').compute()

# Convert the precip values to mm/day
Y["pr"] *= 86400
Y["pr90"] *= 86400

In [ ]:
# Get the test data (NOT visible to contestants)

test_Y = xr.open_dataset(data_path + 'outputs_ssp245.nc').compute()
test_X = xr.open_dataset(data_path + 'inputs_ssp245.nc').compute()


# Input dimensionality reduction

For this baseline example I've decided to only take the leading few EOFs of the aerosol emissions data using the `eofs` package.

In [ ]:
X['SO2'].mean(['latitude', 'longitude']).plot()

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
bc_solver = Eof(X['BC'])

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
bc_eofs = bc_solver.eofsAsCorrelation(neofs=5)
bc_pcs = bc_solver.pcs(npcs=5, pcscaling=1)

for mode, eof  in bc_eofs.groupby('mode'):
    # Plot the leading EOF expressed as correlation in the Pacific domain.
    clevs = np.linspace(-1, 1, 11)
    eof.plot.pcolormesh(subplot_kws={'projection': ccrs.PlateCarree()},
                        cbar_kwargs={'label': '', 'orientation':'horizontal'}, vmin=-1.)
    plt.gca().coastlines()
    plt.gca().set_title(f"EOF{mode} expressed as correlation", fontsize=16)
    plt.show()

In [ ]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
so2_solver = Eof(X['SO2'])

# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
so2_eofs = so2_solver.eofsAsCorrelation(neofs=5)
so2_pcs = so2_solver.pcs(npcs=5, pcscaling=1)

for (mode, eof), (_, pc) in zip(so2_eofs.groupby('mode'), so2_pcs.groupby('mode')):
    # Plot the leading EOF expressed as correlation in the Pacific domain.
    clevs = np.linspace(-1, 1, 11)
    eof.plot.pcolormesh(subplot_kws={'projection': ccrs.PlateCarree()},
                        cbar_kwargs={'label': '', 'orientation':'horizontal'}, vmin=-1.)
    plt.gca().coastlines()
    plt.gca().set_title(f"EOF{mode} expressed as correlation", fontsize=16)
    plt.show()
    pc.plot()
    plt.show()

In [ ]:
# Convert the Principle Components of the aerosol emissions (calculated above) in to Pandas DataFrames
bc_df = bc_pcs.to_dataframe().unstack('mode')
bc_df.columns = [f"BC_{i}" for i in range(5)]

so2_df = so2_pcs.to_dataframe().unstack('mode')
so2_df.columns = [f"SO2_{i}" for i in range(5)]

In [ ]:
# Bring the emissions data back together again and normalise
leading_historical_inputs = pd.DataFrame({
    "CO2": normalize_co2(X["CO2"].data),
    "CH4": normalize_ch4(X["CH4"].data)
}, index=X["CO2"].coords['time'].data)

# Combine with aerosol EOFs
leading_historical_inputs=pd.concat([leading_historical_inputs, bc_df, so2_df], axis=1)

In [ ]:
leading_historical_inputs

# Build baseline model

In [ ]:
from esem import gp_model
from esem.data_processors import Whiten, Normalise

# Just a *very* simple GP with default kernel assuming all years are independant

tas_gp = gp_model(leading_historical_inputs, Y["tas"])
tas_gp.train()

In [ ]:
pr_gp = gp_model(leading_historical_inputs, Y["pr"])
pr_gp.train()

In [ ]:
dtr_gp = gp_model(leading_historical_inputs, Y["diurnal_temperature_range"])
dtr_gp.train()

In [ ]:
pr90_gp = gp_model(leading_historical_inputs, Y["pr90"])
pr90_gp.train()

## Gather the test data

In [ ]:
# Will be hidden from contestants
tas_truth = test_Y["tas"].mean('member')
pr_truth = test_Y["pr"].mean('member') * 86400
pr90_truth = test_Y["pr90"].mean('member') * 86400
dtr_truth = test_Y["diurnal_temperature_range"].mean('member')


In [ ]:
test_inputs = pd.DataFrame({
    "CO2": normalize_co2(test_X["CO2"].data),
    "CH4": normalize_ch4(test_X["CH4"].data)
}, index=test_X["CO2"].coords['time'].data)

# Combine with aerosol EOFs
test_inputs=pd.concat([test_inputs, 
                       bc_solver.projectField(test_X["BC"], neofs=5, eofscaling=1).to_dataframe().unstack('mode').rename(columns={i:f"BC_{i}" for i in range(5)}),
                       so2_solver.projectField(test_X["SO2"], neofs=5, eofscaling=1).to_dataframe().unstack('mode').rename(columns={i:f"_{i}" for i in range(5)}),
                       ], axis=1)

# Evaluate predictions

In [ ]:
m_tas, _ = tas_gp.predict(test_inputs)
m_pr, _ = pr_gp.predict(test_inputs)
m_pr90, _ = pr90_gp.predict(test_inputs)
m_dtr, _ = dtr_gp.predict(test_inputs)

In [ ]:
from matplotlib import colors
divnorm=colors.TwoSlopeNorm(vmin=-2., vcenter=0., vmax=5)

with plt.style.context("dark_background"):
    ax = plt.axes(projection=ccrs.PlateCarree())
    tas_truth.sel(time=2050).plot(cmap="coolwarm", norm=divnorm,
                                  cbar_kwargs={"label":"Temperature change / K"})
    ax.set_title("True 2050")
    ax.coastlines()

In [ ]:
with plt.style.context("dark_background"):
    ax = plt.axes(projection=ccrs.PlateCarree())
    m_tas.sel(sample=35).plot(cmap="coolwarm", norm=divnorm,
                             cbar_kwargs={"label":"Temperature change / K"})
    ax.set_title("Emulated 2050")
    ax.coastlines()

### These are the metrics to be scored on

In [ ]:
def get_rmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return np.sqrt(((truth-pred)**2).weighted(weights).mean(['lat', 'lon'])).data

print(f"RMSE at 2015: {get_rmse(tas_truth[0], m_tas[0])}")
print(f"RMSE at 2050: {get_rmse(tas_truth[35], m_tas[35])}")
print(f"RMSE at 2100: {get_rmse(tas_truth[85], m_tas[85])}")

In [ ]:
print(f"RMSE at 2015: {get_rmse(dtr_truth[0], m_dtr[0])}")
print(f"RMSE at 2050: {get_rmse(dtr_truth[35], m_dtr[35])}")
print(f"RMSE at 2100: {get_rmse(dtr_truth[85], m_dtr[85])}")

In [ ]:
print(f"RMSE at 2015: {get_rmse(pr_truth[0], m_pr[0])}")
print(f"RMSE at 2050: {get_rmse(pr_truth[35], m_pr[35])}")
print(f"RMSE at 2100: {get_rmse(pr_truth[85], m_pr[85])}")

In [ ]:
print(f"RMSE at 2015: {get_rmse(pr90_truth[0], m_pr90[0])}")
print(f"RMSE at 2050: {get_rmse(pr90_truth[35], m_pr90[35])}")
print(f"RMSE at 2100: {get_rmse(pr90_truth[85], m_pr90[85])}")

In [ ]:
def get_rmse(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    return np.sqrt(((truth-pred)**2).weighted(weights).mean(['lat', 'lon', 'sample', 'time'])).data

print(f"RMSE: {get_rmse(tas_truth, m_tas)}")
print(f"RMSE: {get_rmse(dtr_truth, m_dtr)}")
print(f"RMSE: {get_rmse(pr_truth, m_pr)}")
print(f"RMSE: {get_rmse(pr90_truth, m_pr90)}")

